In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

class CausalSelfAttention(nn.Module):

    def __init__(self, config):
        super().__init__()
        assert config['hidden_size'] % config['num_attention_heads'] == 0
        # key, query, value projections for all heads, but in a batch
        self.c_attn = nn.Linear(config['hidden_size'], 3 * config['hidden_size'])
        # output projection
        self.c_proj = nn.Linear(config['hidden_size'], config['hidden_size'])
        self.c_proj.NANGPT_SCALE_INIT = 1
        # regularization
        self.num_attention_heads = config['num_attention_heads']
        self.hidden_size = config['hidden_size']
        self.register_buffer("bias", torch.tril(torch.ones(config['max_position_embeddings'], config['max_position_embeddings'])).view(1, 1, config['max_position_embeddings'], config['max_position_embeddings']))

    def forward(self, x):
        B, T, C = x.size() # batch size, sequence length, embedding dimensionality (hidden_size)
        # calculate query, key, values for all heads in batch and move head forward to be the batch dim
        # nh is "number of heads", hs is "head size", and C (number of channels) = nh * hs
        # e.g. in GPT-2 (124M), num_attention_heads=12, hs=64, so nh*hs=C=768 channels in the Transformer
        qkv = self.c_attn(x)
        q, k, v = qkv.split(self.hidden_size, dim=2)
        k = k.view(B, T, self.num_attention_heads, C // self.num_attention_heads).transpose(1, 2) # (B, nh, T, hs)
        q = q.view(B, T, self.num_attention_heads, C // self.num_attention_heads).transpose(1, 2) # (B, nh, T, hs)
        v = v.view(B, T, self.num_attention_heads, C // self.num_attention_heads).transpose(1, 2) # (B, nh, T, hs)

        att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
        att = att.masked_fill(self.bias[:, :, :T, :T] == 0, float('-inf'))
        att = F.softmax(att, dim=-1)
        y = att @ v # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)

        y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side
        # output projection
        y = self.c_proj(y)
        return y

class TransformerModel(nn.Module):
    def __init__(self, config):
        super(TransformerModel, self).__init__()
        
        # Load model configuration from YAML
        self.hidden_size = config['hidden_size']
        self.vocab_size = config['vocab_size']
        self.num_hidden_layers = config['num_hidden_layers']
        self.num_attention_heads = config['num_attention_heads']
        self.intermediate_size = config['intermediate_size']
        self.max_position_embeddings = config['max_position_embeddings']
        self.rms_norm_eps = config['rms_norm_eps']
        self.tie_word_embeddings = config['tie_word_embeddings']

        # Embedding layers
        self.token_embeddings = nn.Embedding(self.vocab_size, self.hidden_size)
        self.position_embeddings = nn.Embedding(self.max_position_embeddings, self.hidden_size)

        # Transformer layers
        self.layers = nn.ModuleList([
            TransformerLayer(config,self.hidden_size, self.num_attention_heads, self.intermediate_size, self.rms_norm_eps)
            for _ in range(self.num_hidden_layers)
        ])

        # Output layer
        self.lm_head = nn.Linear(self.hidden_size, self.vocab_size)
        if self.tie_word_embeddings:
            self.lm_head.weight = self.token_embeddings.weight

    def forward(self, input_ids):
        # Input embeddings
        seq_length = input_ids.size(1)
        position_ids = torch.arange(0, seq_length, dtype=torch.long, device=input_ids.device)
        position_ids = position_ids.unsqueeze(0).expand_as(input_ids)

        x = self.token_embeddings(input_ids) + self.position_embeddings(position_ids)

        # Pass through Transformer layers
        for layer in self.layers:
            x = layer(x)

        # Final linear layer
        logits = self.lm_head(x)
        return logits

class TransformerLayer(nn.Module):
    def __init__(self, config,hidden_size, num_attention_heads, intermediate_size, rms_norm_eps):
        super(TransformerLayer, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_attention_heads = num_attention_heads
        self.intermediate_size = intermediate_size
        self.head_dim = hidden_size // num_attention_heads

        assert hidden_size % num_attention_heads == 0, "hidden_size must be divisible by num_attention_heads"

        # Self-attention
        self.self_attn = CausalSelfAttention(config)
        self.ln_1 = nn.LayerNorm(hidden_size)

        # Feedforward layers
        self.linear1 = nn.Linear(hidden_size, intermediate_size)
        self.activation = F.silu  # Activation function (SiLU)
        self.linear2 = nn.Linear(intermediate_size, hidden_size)
        self.linear2.NANOGPT_SCALE_INIT = 1

        # Normalization
        self.norm1 = nn.LayerNorm(hidden_size, eps=rms_norm_eps)

    def forward(self, x):
        # Self-attention block
        attn_output = self.self_attn(self.ln_1(x))
        x = x + attn_output  # Residual connection
        x = self.norm1(x)

        # Feedforward block
        ff_output = self.linear2(self.activation(self.linear1(x)))
        x = x + ff_output  # Residual connection

        return x

# Load model configuration from YAML-like dictionary (parsed from YAML)
config = {
    "hidden_size": 576,
    "vocab_size": 49152,
    "num_hidden_layers": 30,
    "num_attention_heads": 9,
    "intermediate_size": 1536,
    "max_position_embeddings": 2048,
    "rms_norm_eps": 1e-5,
    "tie_word_embeddings": True
}


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
#from model import TransformerModel, config
from tokenizers import Tokenizer
from tokenizers.models import WordPiece
from tokenizers.trainers import WordPieceTrainer
from tokenizers.pre_tokenizers import Whitespace
import os
from torchsummary import summary
from tqdm import tqdm
import time

# Function to train a new tokenizer
def train_tokenizer(data_files, vocab_size, save_path):
    """
    Args:
        data_files: List of file paths containing text data.
        vocab_size: Desired vocabulary size.
        save_path: Path to save the trained tokenizer.
    """
    tokenizer = Tokenizer(WordPiece())
    tokenizer.pre_tokenizer = Whitespace()

    trainer = WordPieceTrainer(
    vocab_size=vocab_size,
    special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"],
    min_frequency=1  # Lower threshold to include more tokens
    )

    tokenizer.train(data_files, trainer)

    if not os.path.exists(save_path):
        os.makedirs(save_path)

    tokenizer.save(os.path.join(save_path, "tokenizer.json"))
    return tokenizer

# Function to save a checkpoint
def save_checkpoint(model, optimizer, scheduler, current_batch, checkpoint_path):
    checkpoint = {
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'current_batch': current_batch
    }
    torch.save(checkpoint, checkpoint_path)
    #print(f"Checkpoint saved at batch {current_batch} to {checkpoint_path}")

# Function to load a checkpoint
def load_checkpoint(checkpoint_path, model, optimizer, scheduler):
    if os.path.exists(checkpoint_path):
        checkpoint = torch.load(checkpoint_path, map_location=torch.device(device))
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        current_batch = checkpoint['current_batch']
        print(f"Checkpoint loaded from {checkpoint_path} at batch {current_batch}")
        return current_batch
    else:
        print(f"No checkpoint found at {checkpoint_path}, starting fresh training.")
        return 0

class DataLoaderLite:
    def __init__(self, tokenizer, data_files, B, T):
        self.B = B
        self.T = T

        self.tokenizer = tokenizer

        # Load and tokenize data
        self.data = []
        for file_path in data_files:
            with open(file_path, 'r', encoding='utf-8') as f:
                text = f.read()
                tokens = tokenizer.encode(text).ids
                self.data.extend(tokens)

        print(f'loaded {len(self.data)} tokens')
        print(f'1 epoch = {len(self.data) // (B * T)} batches')

        # state
        self.current_position = 0
    
    def next_batch(self):
        B, T = self.B, self.T
        buf = self.data[self.current_position: self.current_position + B * T + 1]
        
        # Convert list to tensor
        buf = torch.tensor(buf, dtype=torch.long)

        x = buf[:-1].view(B, T)  # inputs
        y = buf[1:].view(B, T)   # targets
        
        # advance the position in the tensor
        self.current_position += B * T

        # if loading the next batch would be out of bounds, reset
        if self.current_position + (B * T + 1) > len(self.data):
            self.current_position = 0

        return x, y

device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    device = "mps"
print(f"using device: {device}")

# SEED
torch.manual_seed(1337)
if torch.cuda.is_available():
    torch.cuda.manual_seed(1337)

data_files = ["/kaggle/input/input1-txt/input.txt"]  # Replace with your text file paths
vocab_size = 49152
tokenizer_save_path = "/kaggle/working/"

# Train a new tokenizer if not already trained
if not os.path.exists(os.path.join(tokenizer_save_path, "tokenizer.json")):
    tokenizer = train_tokenizer(data_files, vocab_size, tokenizer_save_path)
else:
    tokenizer = Tokenizer.from_file(os.path.join(tokenizer_save_path, "tokenizer.json"))

batch_size = 16
tokens_per_batch = 256

train_loader = DataLoaderLite(tokenizer,data_files,B = batch_size, T = tokens_per_batch)

# Function to generate predictions
def generate_predictions(model, tokenizer, text, max_tokens=100, top_k=50):
    model.eval()
    with torch.no_grad():
        input_ids = tokenizer.encode(text).ids
        input_ids = torch.tensor(input_ids, dtype=torch.long, device=device).unsqueeze(0)  # Add batch dimension

        generated_ids = input_ids
        for _ in range(max_tokens):
            outputs = model(generated_ids)
            next_token_logits = outputs[:, -1, :]  # Get logits for the last token
            probs = torch.nn.functional.softmax(next_token_logits, dim=-1)

            if top_k is not None:
                # Top-k sampling
                top_k_probs, top_k_indices = torch.topk(probs, top_k, dim=-1)
                next_token = top_k_indices.gather(
                    dim=-1, index=torch.multinomial(top_k_probs, num_samples=1)
                )
            else:
                # Greedy decoding
                next_token = torch.argmax(probs, dim=-1, keepdim=True)

            # Ensure next_token has the correct shape for concatenation
            next_token = next_token.view(1, 1)  # (batch_size=1, sequence_length=1)

            # Concatenate next_token with generated_ids
            generated_ids = torch.cat([generated_ids, next_token], dim=1)

            # Stop if end-of-sequence token is generated
            if next_token.item() == tokenizer.token_to_id("[SEP]"):
                break

        generated_text = tokenizer.decode(generated_ids.squeeze().tolist())
    model.train()
    return generated_text



config["vocab_size"] = len(tokenizer.get_vocab())
print(f"New Vocab Size is: {config['vocab_size']}")
# Initialize model, optimizer, and scheduler
model = TransformerModel(config)
model.to(device)

def initialize_weights(m):
    if isinstance(m, torch.nn.Linear) or isinstance(m, torch.nn.Embedding):
        torch.nn.init.xavier_uniform_(m.weight)

model.apply(initialize_weights)

# Print the model architecture
print(model)

# Calculate and print the total number of trainable parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total trainable parameters: {trainable_params}")

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.65)

# Define checkpoint path
checkpoint_path = "/kaggle/working/checkpoint.pth"

# Load checkpoint if exists
start_batch = load_checkpoint(checkpoint_path, model, optimizer, scheduler)

# Fixed text for predictions
fixed_text = "He that will give good words to thee will flatter"

# Training loop with tqdm progress bar
total_batches = 5000
checkpoint_interval = len(train_loader.data) // (batch_size * tokens_per_batch)

for i in range(start_batch, total_batches, checkpoint_interval):
    loss_list = []
    start_time = time.time()
    with tqdm(total=checkpoint_interval, desc=f"Training Batches {i}-{i+checkpoint_interval-1}", unit="batch") as pbar:
        for batch_idx in range(checkpoint_interval):
            global_batch = i + batch_idx
            if global_batch >= total_batches:
                break

            x, y = train_loader.next_batch()
            x, y = x.to(device), y.to(device)
            pred = model(x)

            # Reshape predictions and targets for loss computation
            pred = pred.view(-1, pred.size(-1))  # Flatten predictions
            y = y.view(-1)  # Flatten targets

            # Compute loss
            loss = criterion(pred, y)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Update progress bar with current loss
            pbar.set_postfix(loss=loss.item())
            pbar.update(1)
            loss_list.append(loss.item())

            # Generate text and save checkpoint at the last batch of the interval
            if (batch_idx + 1) == checkpoint_interval or global_batch == total_batches - 1:
                generated_text = generate_predictions(model, tokenizer, fixed_text, max_tokens=100)
                print(f"\nGenerated text at batch {global_batch}: {generated_text}")
                save_checkpoint(model, optimizer, scheduler, global_batch, checkpoint_path)

    # Scheduler step
    scheduler.step()
    elapsed_time = time.time() - start_time
    epoch_loss = sum(loss_list) / len(loss_list)
    print(f"Loss: {epoch_loss:.4f}, Time: {elapsed_time:.2f}s")

using device: cuda
loaded 261973 tokens
1 epoch = 63 batches
New Vocab Size is: 19248
TransformerModel(
  (token_embeddings): Embedding(19248, 576)
  (position_embeddings): Embedding(2048, 576)
  (layers): ModuleList(
    (0-29): 30 x TransformerLayer(
      (self_attn): CausalSelfAttention(
        (c_attn): Linear(in_features=576, out_features=1728, bias=True)
        (c_proj): Linear(in_features=576, out_features=576, bias=True)
      )
      (ln_1): LayerNorm((576,), eps=1e-05, elementwise_affine=True)
      (linear1): Linear(in_features=576, out_features=1536, bias=True)
      (linear2): Linear(in_features=1536, out_features=576, bias=True)
      (norm1): LayerNorm((576,), eps=1e-05, elementwise_affine=True)
    )
  )
  (lm_head): Linear(in_features=576, out_features=19248, bias=True)
)
Total trainable parameters: 105384624
No checkpoint found at /kaggle/working/checkpoint.pth, starting fresh training.


Training Batches 0-62: 100%|██████████| 63/63 [00:33<00:00,  1.87batch/s, loss=6.87]


Generated text at batch 62: He that will give good words to thee will flatter : , me ? with . what , , he , she to my he is ' And that good will my , ! know - the me : you as , , , ' , not I for his have not as my : , , the And ' I be , : : to have , the , is , to ? as the for , , is , what , , ! I . ' I , by . . is : you , ? ? the , And of have . : to and by ,


Training Batches 0-62: 100%|██████████| 63/63 [00:37<00:00,  1.67batch/s, loss=6.87]


Loss: 7.2082, Time: 37.68s


...

Training Batches 2772-2834: 100%|██████████| 63/63 [00:34<00:00,  1.85batch/s, loss=6.67]


Generated text at batch 2834: He that will give good words to thee will flatter . with . as thou ' ; me I , , be . him , s do : that ' for . have , : : he : . do , , be , ; , ; , , of , it to I the we I , And , ' the him him s : with : , not : ; , my thee him ' , ? : for : ; - : , : , I thy ; your my your , ' and . , this this I and so the will the of to :


Training Batches 2772-2834: 100%|██████████| 63/63 [00:39<00:00,  1.61batch/s, loss=6.67]


Loss: 6.4365, Time: 39.19s


Training Batches 2835-2897:  65%|██████▌   | 41/63 [00:22<00:11,  1.85batch/s, loss=6.48]

In [7]:
import os
import shutil

# Path to the working folder
working_folder = "/kaggle/working"

# Delete all files and subfolders in the working folder
for filename in os.listdir(working_folder):
    file_path = os.path.join(working_folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)  # Remove file
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)  # Remove folder
    except Exception as e:
        print(f"Failed to delete {file_path}. Reason: {e}")

print("All files and folders in /kaggle/working have been deleted.")

All files and folders in /kaggle/working have been deleted.
